# Retrieve Simple Log Information with Declare4Py

This tutorial will go through the steps necessary to perform a simple analysis of logs with the Declare4Py library.
## Instantiation and simple Utility Functions

Necessary for this tutorial is the `src.declare4py.d4py_event_log.D4PyEventLog` class that contains all the methods for the analysis. For this reason we import `D4PyEventLog` from the `src.declare4py.d4py_event_log` package and the `os` package of python. Then set the path of the log, and instantiate an object of the `D4PyEventLog` class. You have to provide the activity name attribute for the log, the default is `concept:name`.

In [1]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("..", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.declare4py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")

event_log: D4PyEventLog = D4PyEventLog(concept_name="concept:name")

The next step is the parsing of the log with the `parse_xes_log` function. Logs can be passed both in the `.xes` or `xes.gz` formats. 
<br> At the moment we are using the `.xes` parser of PM4PY, which might change in the future. 

In [10]:
# Parses a xes log to EventLog
event_log.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The `event_log` object holds the parsed log, length of the log, frequent set of items and a binary encoding of the log. The last two attributes will be explained in a later paragraph.

Once the log has been successfully parsed, we can get the log itself and its length.

In [11]:
# Print the parsed log
print("This is the log:")
print(event_log.get_log())
print("--------------------------------------")

# Print the number of cases in the log
print("Number of cases:")
print(event_log.get_length())

This is the log:
[{'attributes': {'concept:name': 'A'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': True, 'SIRSCritTachypnea': True, 'Hypotensie': True, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 85, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 10, 22, 11, 15, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': True, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'A', 'Hypoxie': False, 'DiagnosticUrinarySediment': True, 'DiagnosticECG': True}, '..', {'org:group': 'E', 'lifecycle:transition': 'complete', 'concept:name': 'Release A', 'time:timestamp': datetime.datetime(

The `get_trace` function returns a trace given a numeric index.

In [12]:
event_log.get_trace(3)

{'attributes': {'concept:name': 'D'}, 'events': [{'InfectionSuspected': True, 'org:group': 'A', 'DiagnosticBlood': True, 'DisfuncOrg': False, 'SIRSCritTachypnea': True, 'Hypotensie': False, 'SIRSCritHeartRate': True, 'Infusion': True, 'DiagnosticArtAstrup': True, 'concept:name': 'ER Registration', 'Age': 70, 'DiagnosticIC': True, 'DiagnosticSputum': False, 'DiagnosticLiquor': False, 'DiagnosticOther': False, 'SIRSCriteria2OrMore': True, 'DiagnosticXthorax': True, 'SIRSCritTemperature': True, 'time:timestamp': datetime.datetime(2014, 7, 10, 11, 52, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), 'DiagnosticUrinaryCulture': False, 'SIRSCritLeucos': False, 'Oligurie': False, 'DiagnosticLacticAcid': True, 'lifecycle:transition': 'complete', 'Diagnose': 'D', 'Hypoxie': False, 'DiagnosticUrinarySediment': False, 'DiagnosticECG': True}, '..', {'org:group': '?', 'lifecycle:transition': 'complete', 'concept:name': 'Return ER', 'time:timestamp': datetime.datetime(2014, 7, 28, 17, 36

A useful utility function is: `get_log_alphabet_attribute` for retrieving all the values of an attribute in a log.

In [13]:
# Print the set of resources that are in the log
print("Resources names:")
print(event_log.get_log_alphabet_attribute('org:group'))
print("--------------------------------------")

# Print the set of activities that are in the log
print("Activity names:")
print(event_log.get_log_alphabet_attribute('concept:name'))
print("--------------------------------------")

Resources names:
['M', 'N', 'F', 'B', 'I', 'A', 'R', 'W', 'X', 'Q', 'J', 'V', 'C', 'L', 'H', 'K', '?', 'T', 'O', 'P', 'S', 'G', 'E', 'Y', 'U', 'D']
--------------------------------------
Activity names:
['ER Registration', 'Return ER', 'Release D', 'Admission NC', 'ER Triage', 'LacticAcid', 'Release E', 'CRP', 'Admission IC', 'Release B', 'Release A', 'IV Antibiotics', 'ER Sepsis Triage', 'IV Liquid', 'Leucocytes', 'Release C']
--------------------------------------


A log is a complex data structure that can be explored along several dimensions. The function `attribute_log_projection` projects the cases in the log according to the given input attribute. A projection is a list (the log) of lists (the single cases) containing the value of the attribute.

In [ ]:
# Activity projection
for idx, trace in enumerate(event_log.attribute_log_projection("concept:name")):
    print(f"{idx}- {trace}")
print("--------------------------------------")

# Resource projection
for idx, trace in enumerate(event_log.attribute_log_projection("org:group")):
    print(f"{idx}- {trace}")

## Frequent Itemsets

`D4PyEventLog` offers support for computing the frequent itemsets of activities/resources in the log. The function `compute_frequent_itemsets` takes as input the `min_support` of the itemsets, the name of the case id attribute, a list with the names of the attributes you want to discover the itemsets, the `algorithm` to perform the computation (available `fpgrowth` and `apriori`) and `len_itemset` indicating the maximum length of the itemsets, the default is `None`.

In [15]:
frequent_itemsets = event_log.compute_frequent_itemsets(min_support=0.8, case_id_col='case:concept:name', categorical_attributes=['concept:name'], algorithm='fpgrowth', len_itemset=3)
frequent_itemsets

/Users/ivan/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
0,1.000000,(concept:name_ER Triage),1
1,1.000000,(concept:name_ER Registration),1
2,0.999048,(concept:name_ER Sepsis Triage),1
3,0.963810,(concept:name_Leucocytes),1
4,0.959048,(concept:name_CRP),1
5,0.819048,(concept:name_LacticAcid),1
6,1.000000,"(concept:name_ER Triage, concept:name_ER Regis...",2
7,0.999048,"(concept:name_ER Registration, concept:name_ER...",2
8,0.999048,"(concept:name_ER Triage, concept:name_ER Sepsi...",2
9,0.999048,"(concept:name_ER Triage, concept:name_ER Regis...",3
